# Getting Started

To install the most recent stable release for Modin run the following code on your command line:

In [11]:
!pip install "modin[all]" 

For further instructions on how to install Modin with conda or for specific platforms or engines, see our detailed [installation guide](https://modin.readthedocs.io/en/latest/getting_started/installation.html).

Modin acts as a drop-in replacement for pandas so you can simply change a single line of import to speed up your pandas workflows. To use Modin, you simply have to replace the import of pandas with the import of Modin, as follows.

In [ ]:
import modin.pandas as pd
import pandas

In [ ]:
#############################################
### For the purpose of timing comparisons ###
#############################################
import time
import ray
# Look at the Ray documentation with respect to the Ray configuration suited to you most.
ray.init()
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

2024-09-04 02:54:13,335	INFO worker.py:1774 -- Started a local Ray instance. View the dashboard at http://127.0.0.1:8265 


### Dataset: NYC taxi trip data

Link to raw dataset: https://modin-datasets.intel.com/testing/yellow_tripdata_2015-01.csv (**Size: ~200MB**)

In [ ]:
# This may take a few minutes to download
import urllib.request
dataset_url = "https://modin-datasets.intel.com/testing/yellow_tripdata_2015-01.csv"
urllib.request.urlretrieve(dataset_url, "../../data/tmp/taxi.csv")  

('taxi.csv', <http.client.HTTPMessage at 0x24ccdc3f810>)

# Faster Data Loading with Modin's ``read_csv``

In [ ]:
start = time.time()

pandas_df = pandas.read_csv(
    "../../data/tmp/taxi.csv",
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"],
    quoting=3)

end = time.time()
pandas_duration = end - start
print("Time to read with pandas: {} seconds".format(round(pandas_duration, 3)))

Time to read with pandas: 15.494 seconds


In [ ]:
start = time.time()

modin_df = pd.read_csv(
    "../../data/tmp/taxi.csv", 
    parse_dates=["tpep_pickup_datetime", "tpep_dropoff_datetime"], 
    quoting=3)

end = time.time()
modin_duration = end - start
print("Time to read with Modin: {} seconds".format(round(modin_duration, 3)))

printmd("## Modin is {}x faster than pandas at `read_csv`!".format(round(pandas_duration / modin_duration, 2)))

Time to read with Modin: 36.261 seconds


## Modin is 0.43x faster than pandas at `read_csv`!

You can quickly check that the result from pandas and Modin is exactly the same.

In [ ]:
pandas_df

(raylet) The node with node id: 5498cb00a405c68ed01a458dd0104121f12d77fd68da15ca08ac75f7 and address: 127.0.0.1 and node name: 127.0.0.1 has been marked dead because the detector has missed too many heartbeats from it. This can happen when a 	(1) raylet crashes unexpectedly (OOM, etc.) 
	(2) raylet has lagging heartbeats due to slow network or busy workload.


(raylet) [2024-09-04 02:56:24,239 E 26604 28332] (raylet.exe) agent_manager.cc:83: The raylet exited immediately because one Ray agent failed, agent_name = dashboard_agent/15724.
(raylet) The raylet fate shares with the agent. This can happen because
(raylet) - The version of `grpcio` doesn't follow Ray's requirement. Agent can segfault with the incorrect `grpcio` version. Check the grpcio version `pip freeze | grep grpcio`.
(raylet) - The agent failed to start because of unexpected error or port conflict. Read the log `cat /tmp/ray/session_latest/logs/{dashboard_agent|runtime_env_agent}.log`. You can find the log file structure here https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#logging-directory-structure.
(raylet) - The agent is killed by the OS (e.g., out of memory).
(raylet) [2024-09-04 02:56:24,438 E 26604 24240] (raylet.exe) raylet.cc:175: Raylet failed to accept new connection: The I/O operation has been aborted because of either a thread exi

,Unnamed: 0.1,Unnamed: 0,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,...,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,0,0,1,2015-01-01 00:11:33,2015-01-01 00:16:48,1,1.00,1,N,41,...,1,5.7,0.5,0.5,1.40,0.0,0.0,8.40,NaN,NaN
1,1,1,1,2015-01-01 00:18:24,2015-01-01 00:24:20,1,0.90,1,N,166,...,3,6.0,0.5,0.5,0.00,0.0,0.0,7.30,NaN,NaN
2,2,2,1,2015-01-01 00:26:19,2015-01-01 00:41:06,1,3.50,1,N,238,...,1,13.2,0.5,0.5,2.90,0.0,0.0,17.40,NaN,NaN
3,3,3,1,2015-01-01 00:45:26,2015-01-01 00:53:20,1,2.10,1,N,162,...,1,8.2,0.5,0.5,2.37,0.0,0.0,11.87,NaN,NaN
4,4,4,1,2015-01-01 00:59:21,2015-01-01 01:05:24,1,1.00,1,N,236,...,3,6.0,0.5,0.5,0.00,0.0,0.0,7.30,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1933502,1933502,1933502,2,2015-01-06 10:28:20,2015-01-06 10:40:50,1,1.13,1,N,211,...,2,8.5,0.0,0.5,0.00,0.0,0.3,9.30,NaN,NaN
1933503,1933503,1933503,2,2015-01-06 10:54:19,2015-01-06 11:21:37,1,5.39,1,N,87,...,1,20.0,0.0,0.5,4.00,0.0,0.3,24.80,NaN,NaN
1933504,1933504,1933504,2,2015-01-06 10:19:54,2015-01-06 10:24:05,2,0.83,1,N,239,...,1,5.0,0.0,0.5,1.00,0.0,0.3,6.80,NaN,NaN
1933505,1933505,1933505,2,2015-01-06 10:26:46,2015-01-06 10:42:05,3,2.29,1,N,143,...,1,11.0,0.0,0.5,2.20,0.0,0.3,14.00,NaN,NaN


In [ ]:
modin_df

OSError: Unknown error

OSError: Unknown error

# Faster Append with Modin's ``concat``

Our previous ``read_csv`` example operated on a relatively small dataframe. In the following example, we duplicate the same taxi dataset 100 times and then concatenate them together.

Please note that this quickstart notebook is assumed to be run on a machine that has enough memory in order to be able to perform the operations both with pandas and Modin in a single pipeline (which at least doubles the amount of required memory). If your machine doesn't have enough resources to execute every cell of the notebook and you see an OOM issue, you most likely need to reduce ``N_copies`` in the cell below.

In [ ]:
N_copies= 10
start = time.time()

big_pandas_df = pandas.concat([pandas_df for _ in range(N_copies)])

end = time.time()
pandas_duration = end - start
print("Time to concat with pandas: {} seconds".format(round(pandas_duration, 3)))

MemoryError: Unable to allocate 1.44 GiB for an array with shape (1, 193350700) and data type int64

In [ ]:
start = time.time()

big_modin_df = pd.concat([modin_df for _ in range(N_copies)])

end = time.time()
modin_duration = end - start
print("Time to concat with Modin: {} seconds".format(round(modin_duration, 3)))

printmd("### Modin is {}x faster than pandas at `concat`!".format(round(pandas_duration / modin_duration, 2)))

Time to concat with Modin: 0.564 seconds


### Modin is 60.57x faster than pandas at `concat`!

The result dataset is around 19GB in size.

In [ ]:
big_modin_df.info()

(apply_list_of_funcs pid=73415) 
(apply_list_of_funcs pid=73416) 
<class 'modin.pandas.dataframe.DataFrame'>
Int64Index: 136976500 entries, 0 to 1369764
Data columns (total 18 columns):
 #   Column                 Non-Null Count      Dtype         
---  ---------------------  ------------------  -----         
 0   VendorID               127141300 non-null  float64
 1   tpep_pickup_datetime   136976500 non-null  datetime64[ns]
 2   tpep_dropoff_datetime  136976500 non-null  datetime64[ns]
 3   passenger_count        127141300 non-null  float64
 4   trip_distance          136976500 non-null  float64
 5   RatecodeID             127141300 non-null  float64
 6   store_and_fwd_flag     127141300 non-null  object
 7   PULocationID           136976500 non-null  int64
 8   DOLocationID           136976500 non-null  int64
 9   payment_type           127141300 non-null  float64
 10  fare_amount            136976500 non-null  float64
 11  extra                  136976500 non-null  float64
 12  mt

## Faster ``apply`` over a single column

The performance benefits of Modin becomes aparent when we operate on large gigabyte-scale datasets. For example, let's say that we want to round up the number across a single column via the ``apply`` operation. 

In [ ]:
start = time.time()
rounded_trip_distance_pandas = big_pandas_df["trip_distance"].apply(round)

end = time.time()
pandas_duration = end - start
print("Time to apply with pandas: {} seconds".format(round(pandas_duration, 3)))

Time to apply with pandas: 43.969 seconds


In [ ]:
start = time.time()

rounded_trip_distance_modin = big_modin_df["trip_distance"].apply(round)

end = time.time()
modin_duration = end - start
print("Time to apply with Modin: {} seconds".format(round(modin_duration, 3)))

printmd("### Modin is {}x faster than pandas at `apply` on one column!".format(round(pandas_duration / modin_duration, 2)))

Time to apply with Modin: 1.225 seconds


### Modin is 35.88x faster than pandas at `apply` on one column!

# Summary

Hopefully, this tutorial demonstrated how Modin delivers significant speedup on pandas operations without the need for any extra effort. Throughout example, we moved from working with 100MBs of data to 20GBs of data all without having to change anything or manually optimize our code to achieve the level of scalable performance that Modin provides.

Note that in this quickstart example, we've only shown ``read_csv``, ``concat``, ``apply``, but these are not the only pandas operations that Modin optimizes for. In fact, Modin covers [more than 90% of the pandas API](https://github.com/modin-project/modin/blob/main/README.md#pandas-api-coverage), yielding considerable speedups for many common operations.